# マニュアル管理サービス開発

このノートブックは、マニュアル管理サービスの開発を行うためのノートブックです。  
開発を通して、Ruby on Rails/Vue.jsの基本的な使い方を学びます。

## セットアップ

1. `npm i && bundle install`

## 1. モデル作成

マニュアルや関連するモデルを作成します。  
最低限の情報としてタイトル、内容を持つマニュアルと、その配下に各ステップを紐づけます。

```
bin/rails g model Manual title:string description:text
bin/rails g model Step title:string description:text manual:references
bin/rails db:create db:migrate
```

モデルが作成されたかどうかを確認します。  
以下のコマンドを実行し、`Manual`モデルのレコード数が0であることを確認します。

1. `bin/rails runner 'p Manual.count'`

また、`Manual`モデルに`has_many`を設定し、ManualからStepを取得できるようにします。

```ruby
# app/models/manual.rb
class Manual < ApplicationRecord
  has_many :steps, dependent: :destroy
end
```

## 2. ダミーデータの作成

開発を進めるために、ダミーデータを作成します。  
今回は[faker](https://github.com/faker-ruby/faker)というGemを利用し、ダミーデータを作成します。  
まずは、Fakerをインストールします。

```sh
bundle add faker --group development,test
```

ダミーデータを作成するために、`db/seeds.rb`に以下のコードを追加します。

```ruby
# db/seeds.rb
if Manual.count.zero?
  50.times do
    Manual.create!(
      title: Faker::Book.title,
      description: Faker::Lorem.paragraph
    ).tap do |manual|
      [10, 20, 30].sample.times do
        manual.steps.create!(
          title: Faker::Book.title,
          description: Faker::Lorem.paragraph
        )
      end
    end
  end
end
```

データがない場合にのみデータを作成するようにしていることに注意してください。  
以下のコマンドを実行し、ダミーデータが作成し、`Manual`モデルのレコード数が50であることを確認します。

```
bin/rails db:seed
bin/rails runner 'p Manual.count'
```

## 3. APIの作成

マニュアル一覧・詳細を取得するためのAPIを作成します。  
今回は[graphql-ruby](https://graphql-ruby.org/)を利用してGraphQL APIを作成します。  
まずは、`graphql-ruby`をインストールします。

```sh
bundle add graphql
bin/rails g graphql:install
```

次にルーティングを設定します。  
以下のように書き換えてください。

```ruby
# config/routes.rb
Rails.application.routes.draw do
  post "/graphql", to: "graphql#execute"
end
```

次に`Manual`と`Step`に対するTypeを作成します。  
`graphql-ruby`は、`db/schema.rb`を元にTypeを作成するため、以下のコマンドを実行します。

```sh
bin/rails g graphql:object Manual
bin/rails g graphql:object Step
```

このままだと`Manual`に紐づく`Step`を取得できなため、`ManualType`に`steps`フィールドを追加します。

```ruby
# app/graphql/types/manual_type.rb
module Types
  class ManualType < Types::BaseObject
    #　省略
    field :steps, [Types::StepType], null: true
    
    def steps
      object.steps
    end
  end
end
```

次に、`Manual`と`Step`に対するQueryを作成します。  
`Types::QueryType`に`manuals`と`manual`を追加します。

```ruby
# app/graphql/types/query_type.rb
module Types
  class QueryType < Types::BaseObject
    # 省略
    
    field :manuals, [Types::ManualType], null: false
    def manuals
      Manual.all
    end

    field :manual, Types::ManualType, null: true do
      argument :id, ID, required: true
    end
    def manual(id:)
      Manual.find(id)
    end
  end
end
```

これで準備は完了です。  
以下のコマンドを実行し、`bin/dev`でサーバーを立ち上げ、Thunder Clientなどのツールを使ってAPIを叩いて動作を確認をします。

```graphql
query {
  manuals {
    id
    title
    description
    steps {
      id
      title
      description
    }
  }
}
```

プロパティを追加したり、削除したりして、レスポンスの内容が変わることを確認してください。

## 4. フロントエンドの作成

Vue.jsを使ったフロントエンドを作成します。  
コンポーネントはいくつか用意してあるので、一覧画面と詳細画面を作成します。

まずは、GraphQL用のクエリを定義します。

```typescript
// src/graphql/queries.ts
import gql from 'graphql-tag'

export const GET_MANUALS = gql`
  query GetManuals {
    manuals {
      id
      title
      description
      steps {
        id
      }
    }
  }
`

export const GET_MANUAL = gql`
  query GetManual($id: ID!) {
    manual(id: $id) {
      id
      title
      description
      steps {
        id
        title
        description
      }
    }
  }
`
```

次に一覧画面を編集して、マニュアル一覧を実装していきます。
`src/views/HomeView.vue`を以下のように編集してください。

```vue
<script setup lang="ts">
import { onMounted, ref } from 'vue'
import { useQuery } from '@vue/apollo-composable'
import { GET_MANUALS } from '@/graphql/queries'

const manuals = ref<Manual[]>([])

onMounted(async () => {
  useQuery(GET_MANUALS).onResult(({ data }) => {
    manuals.value = data.manuals
  })
})
</script>

<template>
  <v-container>
    <v-row>
      <v-col v-for="manual in manuals" :key="manual.title" cols="12">
        <v-card @click="$router.push(`/manuals/${manual.id}`)">
          <v-card-title>{{ manual.title }}</v-card-title>
          <v-card-text>{{ manual.description }}</v-card-text>
          <v-card-actions>
            <v-btn>{{ manual.steps.length }} steps</v-btn>
          </v-card-actions>
        </v-card>
      </v-col>
    </v-row>
  </v-container>
</template>
```

次は、詳細画面を作成します。  
`src/views/ManualView.vue`を以下のように編集してください。

```vue
<script setup lang="ts">
import { onMounted, ref } from 'vue'
import { useRoute } from 'vue-router'
import { useQuery } from '@vue/apollo-composable'
import { GET_MANUAL } from '@/graphql/queries'

const manual = ref<Manual | null>(null)
const route = useRoute()

onMounted(() => {
  const { id } = route.params
  useQuery(GET_MANUAL, { id: id }).onResult(({ data }) => {
    manual.value = data.manual
  })
})
</script>

<template>
  <v-container>
    <div v-if="!manual">Loading...</div>
    <div v-if="manual">
      <h1>{{ manual.title }}</h1>
      <p>{{ manual.description }}</p>
      <v-row :style="{ marginTop: '20px' }">
        <v-col
          v-for="(step, index) in manual.steps"
          :key="step.title"
          cols="12"
        >
          <v-card>
            <v-card-title>STEP-{{ index + 1 }}: {{ step.title }}</v-card-title>
            <v-card-text>{{ step.description }}</v-card-text>
          </v-card>
        </v-col>
      </v-row>
    </div>
  </v-container>
</template>
```

最後に、ルーティングを設定します。  
`router`の設定を以下のように変更してください。

```typescript
// src/router/index.ts
const router = createRouter({
  history: createWebHistory(import.meta.env.BASE_URL),
  routes: [
    {
      path: '/',
      name: 'home',
      component: HomeView,
    },
    {
      path: '/manuals/:id',
      name: 'manual-view',
      component: ManualView,
    },
  ],
})
```

これで準備は完了です。  
以下のコマンドを実行し、`bin/dev`でサーバーを立ち上げ、ブラウザでアプリケーションを開いて動作を確認してください。

## 5. おまけ

### N+1問題の解消

現在の実装では、マニュアル一覧を取得する際に、各マニュアルに紐づくステップも取得しています。  
ログを見てみると、マニュアルごとにステップを取得しているため、N+1問題が発生していることが分かります。  
これはGraphQLだけの問題ではなく、RESTful APIでも発生する問題です。

今回は、GraphQLを使っているので`DataLoader`を使って解消します。
既に作成してある`Sources::Association`を使って、`Types::ManualType`に`steps`フィールドを変更します。

```ruby
# app/graphql/types/manual_type.rb
module Types
  class ManualType < Types::BaseObject
    # 省略
    field :steps, [Types::StepType], null: true # ここは変更なし

    def steps
      dataloader.with(Sources::Association, :steps, ::Manual.preload(:steps)).load(object)
    end
  end
end
```

書き換え後に再度APIを叩いて、ログを確認してください。  
そうすると、マニュアル一覧のクエリとステップのクエリが1回ずつ発行されていることが分かります。

これでN+1問題が解消されました。  
このようにN+1問題はデータベースのクエリを最適化することで解消できます。
